In [ ]:
#!git clone https://github.com/neuralcomputer/ML_School.git

# Тема 29. Обучение с подкреплением.

<p align="right"> <i> Не пора ли подкрепиться </i> </p>

## Введение

Мы уже знаем два вида обучения - обучение с учителем и обучение без учителя.
 
В обучении с учителем учитель говорит нам как должно быть, какой должен быть выход при заданном входе. И мы учимся имитировать учителя: а что бы он сказал, если вход такой-то.

Но что, если "учитель" сам не знает, как должно быть? Но зато знает "что такое хорошо, и что такое плохо". 

Представьте, что ваш школьный учитель спрашивает у вас "сколько лет длилась [Столетняя война](https://ru.wikipedia.org/wiki/Столетняя_война) ?", а вы не знаете, но понимаете, что что-то около 100 лет. И говорите: "Столетняя война длилась сто лет!", а учитель говорит, "нет, не правильно, садись два". На другой день вы подумали и говорите: "Столетняя война длилась сто десять лет!", а учитель говорит, "нет, не правильно, садись два", и так повторяется много-много раз. Вы говорите 90 лет - неправильно, говорите 120 лет - неправильно. Весь журнал уже в двойках, надежды нет, но наконец вы говорите: "Столетняя война длилась сто шестнадцать лет!" и, о чудо, учитель говорит "правильно, пятерка, все двойки стираю". 

В этом дурацком примере (а я надеюсь у вас нет таких учителей в школе:) ) учитель не говорил вам, какой ответ, но вы смогли его угадать по реакции учителя, перебирая возможные варианты и получая штраф от учителя. И в итоге решили задачу!

Конечно, для обучения людей это не очень хороший способ, но для компьютеров вполне применим. Ведь часто мы сами не знаем, как должно быть, но можем оценить, как правильно, а как не правильно.

Представьте, вы делаете игру в тетрис, и хотите, чтобы компьютер мог сам играть. Сложно, да и не нужно, прописать все-все "правильные" действия для каждой фигуры, но зато мы знаем, если появились "дырки" это плохо, а если ряд исчез, это очень хорошо. Будем штрафовать компьютер за "дырки" и награждать за исчезновения ряда. Тогда можно заставить компьютер подбирать такие действия, которые приводили бы к максимальной награде за всю игру. 

Такой способ обучения назвали **обучение с подкреплением** (по-английски reinforcement learning) и сегодня он часто используется, например, для создания ботов для игр.

Итак, суть обучения с подкреплением состоит в том, что:
- есть некий игрок, его называют *агент* (agent), который может совершать некоторые действия (actions) и изменять свое состояние (state) в некоторой *среде* (environment).
- за совершение действий агент получает штраф\награду (reward). При совершении действия состояние агента меняется.  
- агент стремится увеличить суммарную награду, полученную за все время. 
- цель агента - найти такую последовательность действий, которая приводит к максимальной суммарной награде (или, что то же, к минимальному штрафу).  

![img](https://drive.google.com/uc?id=1EbI-YVpjIDyZz3otw2nnZljkcYhYiZJt)

В примере с тетрисом, состояние - это поле, заполненное фигурами и положение падающей фигуры, а действия: повороты, перемещения и сброс фигуры. За исчезновение рядов получаем награду - игровые очки - и стремимся набрать как можно больше.   


# Q-обучение
Придумано много разных вариантов обучения с подкреплением, но, пожалуй, самым известным из них является Q-обучение (Q-learning).

Давайте представим, что агент совершает действие **a**, изменяет состояние **s**, получает награду (или штраф) **r**. Мы можем ввести некоторую функцию **Q(s,a)**, которая зависит от состояния и действия, и показывает, а какой же доход (суммарную награду) в итоге, в конце игры, получит агент, если начнет игру, находясь в состоянии **s** совершит действие **a** и продолжит игру до конца.

Если бы такая функция нам была известна, то все было бы просто - выбирай такие действия, которые максимизируют доход и выигрывай. Но, конечно, мы не знаем такую функцию изначально. Надо ее узнать!

Представим, что такая функция все же есть. Как она будет отличаться на разных этапах нашей игры?

Находясь в некотором состоянии **s** агент выполнил действие **a**, перешел в новое состояние **s1** и получил награду **r**. В новом состоянии его функция **Q(s1,a1)** отличается от функции **Q(s,a)** в состоянии **s**  только на величину награды, ведь ее мы и добавили. Если мы знаем значение Q для одного состояния и знаем награды за  переход между состояниями, то можем посчитать Q и для всех других состояний.

Реальность несколько сложней, часто бывает так, что, находясь в одном и том же состоянии, совершая то же самое действие, агент попадает в другое состояние **s2** (пример: действие "выстрел" приводит к разным состояниям, в зависимости от того попал или не попал игрок в цель). Переходы между состояниями - случайный процесс. Поэтому мы не можем напрямую посчитать значения Q, а должны находить их "в среднем" для большого числа примеров.

Для того чтобы на лету считать среднее (точней приближаться к нему) используют такой подход: пусть величина была **Q(s,a)**, а должна то быть **r+Q(s1,a1)** - это число разное в разных примерах (эпизодах) игры.

Давайте изменим величину **Q(s,a)** в сторону, чтобы приблизить ее к **r+Q(s1,a1)** но не полностью, на некоторую долю **L**. Простая геометрия, есть две точки, начало и конец отрезка, конец движется, а мы смещаемся на небольшую часть этого отрезка. К слову, это чем-то напоминает работу зенитной ракеты, цель движется, а ракета постепенно, маленькими шажками, приближается к ней, постоянно следя за целью.

Эту идею можно записать математически:
$Q^{new}(s,a) \leftarrow Q(s,a)+ L*(Q(s_1,a_1)+r - Q(s,a))$

или, вынеся за скобку

$Q^{new}(s,a) \leftarrow (1-L)*Q(s,a)+ L*(Q(s_1,a_1)+r)$ 

Обычно, доход, который мы получим в далеком будущем менее значим, чем доход, который мы получим сейчас. Это можно учесть в нашей формуле введя дополнительный коэффициент **g** - коэффициент "забывания" дохода (по-английски discount, скидка) - число от 0 до 1.

$Q^{new}(s,a) \leftarrow (1-L)*Q(s,a)+ L*(g*Q(s_1,a_1)+r)$ 

Мы получили формулу для известного метода [SARSA](https://en.wikipedia.org/wiki/State–action–reward–state–action). Можем еще уточнить ее.

Нет гарантий, что действие 𝑎1 из 𝑄(𝑠1,𝑎1) лучшее, оно просто произошло. Но раз мы имеем функцию 𝑄, то можем выбрать "наилучшее" действие, такое, для которого значение 𝑄 максимально среди всех возможных действий. Так получается формула для метода 𝑄-обучения.

$Q^{new}(s,a) \leftarrow (1-L)*Q(s,a)+ L*(g*max_a[Q(s_1,a)]+r)$ 


Подытожим:
- агент может совершать действия, менять состояние и получать награду.
- агент оценивает функцию 𝑄. Изначально она задается случайно, а затем по выведенным формулам изменяется. По сути, 𝑄 это таблица, где для всех состояний и действий записан возможный будущий доход.
- на каждом шаге агент выбирает действие и совершает его. 
- в итоге, играя много-много раз, агент настроит все значения из 𝑄, исходя из которой и будут выбираться действия.


## Эпсилон-жадный выбор
Функция 𝑄 показывает нам будущий доход, выбираем действия, которые приводят к максимальному доходу, и мы победили! Такой выбор называют "жадным".

Но на практике возникают сложности, если мы *всегда* выбираем лучшее, с точки зрения 𝑄, действие, то пока эта функция не настроена, может получиться так, что реально "лучшее" и посчитанное лучшее действия разные.
(Уперлись в стенку и лучше ничего не можем придумать, хотя рядом есть дверь). Жадный выбор ограничивает возможности "разведки" состояний, может быть, если мы перейдем в другое, менее ценное состояние, потом из него откроется путь с большим доходом. Да ведь это и в жизни так, что толку Красной Шапочке идти напролом через лес к бабушке и нести ей пирожки, рисковать быть съеденой волком, если можно пройти сотню метров в другую сторону (удаляемся от бабушки) и сесть на маршрутку до бабушкиного дома. Но если Красная Шапочка не знала, что маршрутка есть, поскольку никогда в другую сторону не ходила, то и не узнает.

Если мы при симуляции агента не попадали в некоторые состояния, то и не сможем настроить функцию 𝑄 для них, а вдруг там лучшее решение? Значит надо разрешить агенту, хотя бы в теории, посещать все состояния. Это можно с делать с помощью "эпсилон-жадного" выбора.

Давайте выберем некоторое маленькое число эпсилон от 0 до 1. На каждом шаге симуляции нашего агента возьмем случайное число от 0 до 1. Сравним его с эпсилон. Если больше - используем "жадный" выбор действия. Но если меньше - то выбираем действие случайно. Таким образом, мы допускаем иногда совершение случайных действий, и это дает надежду, что, хоть и при большом числе попыток, мы посетим все состояния. Но такие случайные действия совершаются редко, мы регулируем это величиной числа эпсилон. Отсюда и название способа.

Если эпсилон большое - то мы почти всегда совершаем случайные действия - ничему не научимся. Если очень маленькое - то выбираем "жадно" и можем пропустить важные состояния. Надо искать баланс, и, как всегда, это придется делать интуитивно, подбором. 




# Пример управления тележкой
Что ж, перейдем к примеру.

Есть одномерная тележка (машинка), стоящая под горой. У тележки есть слабенький двигатель, силы которого не хватает, чтобы заехать в горку. А нужно доехать до флажка на горе. В этом мире действует гравитация, поэтому тележка может заехать в горку, если правильно раскачается, совсем как на качелях.

Необходимо определить, как прикладывать силу двигателя, чтобы успешно раскачаться и заехать на гору. Ну-ка, сможете ли вы решить такую задачу только физикой? Попробуйте, а мы решим ее с помощью обучения с подкреплением.

**Состояние** тележки (все единицы измерения условные):
- координата x от -1.2 до + 0.6 
- скорость тележки от -0.07 до + 0.07

**Действия**:
- 0 - прикладывать силу влево
- 1 - не прикладывать силу
- 2 - прикладывать силу вправо

**Награда**:
- награда (штраф) "-1" за каждый шаг симуляции.
- награда 0 за достижение цели.

Прочие параметры среды:
- гравитация: 0.0025
- сила двигателя: 0.001
- вид горы - синусоида 0.45*sin(3x)+0.55
- положение флажка-цели: 0.5 по x координате, скорость 0 
- начальное положение задается случайно в интервале [-0.6 , -0.4], скорость нулевая.

Тележка движется по горе под действием силы тяжести и двигателя, трения нет. Движение симулируется по маленьким отрезкам времени, шагам, если число шагов превысило 200 или тележка заехала за флажок (координата больше 0.5), то симуляция останавливается.

Каждый эпизод - это симуляция движения тележки по шагам.


![img](https://drive.google.com/uc?id=1PpUaur-60cvruEI8r_ifX5ohoWE_UwN1)




## библиотека `gym`
Как вы должно быть заметили, самое сложное технически это именно симуляция среды и агента, а сам метод Q-обучения довольно прост. 

Для симуляции такой тележки воспользуемся библиотекой [`gym`](https://gym.openai.com/envs/MountainCar-v0/).


К сожалению, в Colab, в котором мы работаем, есть проблемы с отображением анимации, которую может рисовать библиотека gym. Поэтому нам придется сделать некоторые неочень понятные действия, чтобы можно было посмотреть анимацию. Вкратце, мы сделаем *виртуальный* дисплей, на котором будет отображена анимация, а на самомо деле запишем ее в видео-файл. Но не вникайте слишком глубоко, просто выполните код в ячейках ниже.
Нами будет сделана функция show_video(), которая и будет показывать записанное видео с этого виртуального дисплея. Чтобы указать, что будет записываться в этот виртуальный дисплей сделаем функцию wrap_env(). 

Если вы работаете в отдельном Jupyter Notebook, а не в Colab, то этого делать ненадо.


In [1]:
# установим библиотеки для виртуального дисплея  
!apt-get install xvfb
!pip install pyvirtualdisplay xvfbwrapper

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 13 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.8 [784 kB]
Fetched 784 kB in 2s (372 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 149406 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.8_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.8) ...
Setting up xvfb (2:1.19.6-1ubuntu4.8) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
  Created wheel for xvfbwrapper: filename=xvfbwrapper-0.2.9-cp37-none-any.whl size=5009 sha256=c208c30c061e05b85f44dfd17f8e6e1abfe17f5aa55f77105e124a91d326e246
  Stored in directory: /root/.cache/pip/wheels/10/f2/61/cacfaf8

In [2]:
# подключим библиотеку для симуляций среды
import gym
import numpy as np #
# подключим вспомогательные библиотеки для виртуального дисплея и отображения анимации
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
# сделаем виртуальный дисплей
display = Display(visible=0, size=(1400, 900))
display.start()



In [3]:
"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""
# функция для показа видео  
def show_video(last=False):
  mp4list = glob.glob('video/*.mp4')
  mp4list.sort()
  if len(mp4list) > 0:
    if last:
      mp4list=mp4list[-1:]

    for mp4 in mp4list:
      #mp4 = mp4list[0]
      video = io.open(mp4, 'r+b').read()
      encoded = base64.b64encode(video)
      print(mp4)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                  loop controls style="height: 400px;">
                  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    
# функция для наблюдения за виртуальным дисплеем 
def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

Библиотека дает возможность работать с разными симуляциями, наша носит название "MountainCar-v0". Подключим библиотеку, зададим методом make() выбранную симуляцию.

Создается объект env с полями и методами, которые увидим позже.


In [4]:
# сделаем папку для сохранения найденного решения 
!mkdir qtables

In [5]:
# 
import gym # подключим библиотеку для имитации внешней среды
import numpy as np #
#env = gym.make("MountainCar-v0") # выберем внешнюю среду - машинка на горке - для Jupyter Notebook
env = wrap_env(gym.make("MountainCar-v0")) # выберем внешнюю среду - машинка на горке - для Colab

Зададим необходимые параметры, константы.

In [6]:
LEARNING_RATE = 0.1 # шаг обучения, (L в формулах)

DISCOUNT = 0.95 # параметр забывания (g в формулах)
EPISODES = 10000 # количество эпизодов
SHOW_EVERY = 1000 # как часто показывать анимацию

Состояний, если их измерять в действительных числах, очень много (ну-ка, посчитайте сколько точек находится между концами отрезка?). Этого и не надо, давайте разобьем все состояния (координата и скорость тележки) на интервалы и будем считать состоянием номер такого интервала. Так мы сами регулируем число состояний. Такой процесс называется "дискретизацией", мы делаем состояние "дискретной" величиной, что очень удобно для компьютеров.    

In [7]:
# дискретизируем состояния
DISCRETE_OS_SIZE = [20, 20] # количество состояний для обеих переменных
discrete_os_win_size = (env.observation_space.high - env.observation_space.low)/DISCRETE_OS_SIZE # величина окна дискретизации

# дискретизация состояния
def get_discrete_state(state):
    # получаем состояние от среды, преобразуем в номер дискрета (интервала).
    discrete_state = (state - env.observation_space.low)/discrete_os_win_size #
    return tuple(discrete_state.astype(np.int))  # преобразуем в целый тип

Зададим параметры для эпсилон-жадного выбора. Будем уменьшать эпсилон на некоторую величину в течении некоторого количества эпизодов. Значит сначала эпсилон будет большое - случайные действия, а затем уменьшится - будем больше полагаться на "лучший" выбор действий.

In [8]:
# Параметры для случайных действий
epsilon = 1  # начальное значение
START_EPSILON_DECAYING = 1 # эпизод, с которого начинается изменение параметра
END_EPSILON_DECAYING = EPISODES//2 # эпизод, на котором заканчивается изменение параметра
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING) # на сколько изменяем

Сделаем Q-функцию в виде трехмерного массива, первые два измерения - состояния, третье - действия (их три штуки). Изначально заполним массив случайными числами от -2 до 0.

In [9]:
# Q-таблица состояние-действие. Представлена трехмерным массивом [20,20,3]
# первые два измерения - состояния, третье - действия, их три штуки.
q_table = np.random.uniform(low=-2, high=0,
                            size=(DISCRETE_OS_SIZE + [env.action_space.n])) #
q_table.shape

(20, 20, 3)

Начинаем симуляцию эпизодов.

Перед началом каждого эпизода сбрасываем reset() среду, т.е. начинаем всегда одинаково. 

Для каждого эпизода, пока не достигнем его конца будем:
- выбирать действие action эпсилон-жадным методом
- совершать выбранное действие, метод step(), среда вернет новое состояние (не забудем и его дискретизировать new_discrete_state), награду, скажет закончен ли эпизод.
- если эпизод еще не закончен, то применяем Q правило для этих состояния и действия, т.е. обновляем соответствующий элемент массива q_table. Из-за того, что мы решили состояние определять *номером* интервала, его же можно использовать как индекс массива. 
- если эпизод закончен и цель достигнута (проверяем, сравнивая координату тележки и координату цели env.goal_position ), то надо дать награду по-больше. В примере это имитируется заданием значения 0 для соответствующего элемента массива. Выходим из эпизода
- изменяем состояние на новое и повторяем.

В конце эпизода - изменяем эпсилон, если надо.

Будем иногда рисовать анимацию в отдельном окне, env.render(), но не очень часто, это долгий процесс и, честно говоря, не очень хорошо сделанный в данной библиотеке. В конце вызовем env.close() для закрытия окна симуляции.

Запаситесь терпением, обучение не быстрый процесс.


In [10]:
# циклы по эпизодам
for episode in range(EPISODES): #
    discrete_state = get_discrete_state(env.reset()) # сбрасываем состояние и получаем дискретное состояние
    done = False # флаг, показывающий закончен ли эпизод
    #
    if episode % SHOW_EVERY == 0: # каждые SHOW_EVERY эпизодов будем рисовать анимацию
        render = True # флаг, что будем рисовать анимацию
        print(episode) # печатаем номер эпизода
    else:
        render = False # а иначе не будем рисовать анимацию (анимация это долго!)
    # в цикле, пока эпизод не закончен
    while not done:
        # эпсилон-жадная стратегия: выбираем случайное число
        if np.random.random() > epsilon: # если оно больше эпсилон
            # выбираем действие для этого состояния, для которого значение таблицы Q максимально 
            action = np.argmax(q_table[discrete_state]) 
        else:
            # а иначе выбираем действие случайно
            # env.action_space.n - число возможных действий
            action = np.random.randint(0, env.action_space.n) #

        # совершаем выбранное действие, получаем новое состояние, награду, флаг об окончании эпизода
        new_state, reward, done, _ = env.step(action) #
        # print(new_state, reward, done)
        # новое состояние тоже дискретизируем
        new_discrete_state = get_discrete_state(new_state) #

        if episode % SHOW_EVERY == 0:   #         
            env.render() # рисуем анимацию
                        
            
        # Если эпизод еще не закончен
        if not done: #

            # находим максимальное значение Q для нового состояния
            max_future_q = np.max(q_table[new_discrete_state]) #

            # текущее значение Q для текущего состояния и выбранного действия
            current_q = q_table[discrete_state + (action,)] #

            # новое значение Q для текущего состояния и выбранного действия - Q-правило
            # учитываем награду, шаг обучения, параметр забывания
            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q) #

            # обновляем это значение в массиве Q
            q_table[discrete_state + (action,)] = new_q #


        # Если эпизод закончен и
        # положение машинки (первая переменная с индексом 0) в новом состоянии
        # больше или равно чем координата флажка финиша (т.е. доехали до него)
        # следует дать награду побольше (ранее мы штрафовали на -1), обновим Q таблицу для такого случая
        elif new_state[0] >= env.goal_position:  
            #q_table[discrete_state + (action,)] = reward
            q_table[discrete_state + (action,)] = 0
            

        discrete_state = new_discrete_state # обновляем текущее состояние

    # Уменьшаем эпсилон, если номер эпизода в указанном диапазоне
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING: #
        epsilon -= epsilon_decay_value #
    # будем сохранять на диск Q таблицу каждые 10 эпизодов.
    if episode % 10 == 0:
        np.save(f"qtables/{episode}_qtable.npy", q_table)

#env.close() # закрываем симуляциию



0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [11]:
show_video() # показываем видео

video/openaigym.video.0.59.video000000.mp4


video/openaigym.video.0.59.video000001.mp4


video/openaigym.video.0.59.video000008.mp4


video/openaigym.video.0.59.video000027.mp4


video/openaigym.video.0.59.video000064.mp4


...

video/openaigym.video.0.59.video005000.mp4


video/openaigym.video.0.59.video006000.mp4


video/openaigym.video.0.59.video007000.mp4


video/openaigym.video.0.59.video008000.mp4


video/openaigym.video.0.59.video009000.mp4


Обычно хватает 5000 эпизодов, чтобы научиться раскачиваться, но помните, что это все же случайный процесс, лучше побольше.

Просимулируем тележку с последней найденной Q-функцией.

In [12]:
i=9990 #
q_table = np.load(f"qtables/{i}_qtable.npy") # загружаем Q-функцию
discrete_state = get_discrete_state(env.reset()) # сбрасываем симуляцию
done=False #
actions=[]; #
while not done:
    action = np.argmax(q_table[discrete_state])  # выбираем действие жадно 
    actions.append(action) # добавляем в историю
    
    new_state, reward, done, _ = env.step(action) # выполняем действие
    new_discrete_state = get_discrete_state(new_state) # дискретизируем новое состояние
    
    discrete_state = new_discrete_state # изменяем состояние
 


In [13]:
show_video(last=True)

video/openaigym.video.0.59.video010000.mp4


In [ ]:
   
Act_name=np.array(('Left','No','Right')) #   названия действий 
Act_name[actions]     # выводим на экран

array(['No', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left',
       'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left',
       'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left',
       'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'No', 'No',
       'No', 'No', 'No', 'No', 'Right', 'Right', 'Right', 'Right', 'No',
       'No', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Right', 'No', 'Right', 'Right', 'Right', 'No', 'Right', 'Right',
       'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right',
       'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Right',
       'Left', 'Left', 'Left', 'No', 'Left', 'No', 'Left', 'Left', 'Left',
       'Left', 'Left', 'Left', 'Left', 'No', 'No', 'Left', 'Left', 'Left',
       'Right', 'No', 'Right', 'Right', 'Left', 'Left', 'Left', 'No',


In [14]:
env.close()#  закроем симуляцию

# Задания
Найденное решение вполне логично, сначала отъезжаем насколько можем влево, затем катимся вправо, меняем на лево, затем на право и этого хватило, чтобы заехать на гору. Мелкие недостатки, что часть времени двигатель не работает, не существенны и не мешают нам. 

А сравните это с плохой, еще не обученной стратегией, скажем для 10 эпизодов.

Посмотрите, как влияют параметры на процесс обучения. Измените эпсилон, измените шаг, и фактор забывания, поменяйте количество дискретов (интервалов) состояния.

# Заключение
Обучение с подкреплением - интереснейшая область машинного обучения. Уже сделаны многие симуляторы, агенты. Так, компьютер обыграл человека в игру Го, играет лучше человека в Старкрафт и другие игры, роботы обучаются и управляются таким методом и др. Но, конечно, есть и недостатки. Обучение с подкреплением требует очень много симуляций, что долго, дорого и не гарантирован результат.

Лучшие результаты достигаются, когда соединяют обучения с подкреплением и нейронные сети, например, нейронная сеть может выступать в роли Q-функции, и тогда не надо изучать вообще все состояния, нейронная сеть аппроксимирует недостающие. 

Простой онлайн-пример можно найти [тут](https://cs.stanford.edu/people/karpathy/convnetjs/demo/rldemo.html).

Агент "видит" объекты, попадающие в лучи, и стремится поедать только красные яблочки, избегая зеленых и стен, но не видит всего лабиринта.

Внутри агента работает простая нейронная сеть, типа многослойного персептрона. В итоге агент научается не врезаться в стены, двигаться в разные стороны, чтобы пройти по лабиринту и не поедать зеленые яблочки.


## Ссылки

Использованы и адаптированы материалы:

https://pythonprogramming.net/q-learning-algorithm-reinforcement-learning-python-tutorial/?completed=/q-learning-reinforcement-learning-python-tutorial/ 

https://gym.openai.com/docs/#getting-started-with-gym 

https://cs.stanford.edu/people/karpathy/convnetjs/demo/rldemo.html 




